In [2]:
import pandas as pd

def ReadSheet(file_path):
    try:
        # Đọc dữ liệu từ sheet 'TKBLop2b', range C8:U38
        df = pd.read_excel(file_path, sheet_name="TKBLop2b", usecols="C:U", skiprows=7, nrows=69)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except ValueError as e:
        print(f"Error reading Excel file: {e}")
        return []
    except Exception as e:
        print(f"Unexpected error: {e}")
        return []

    merged_data = []
    for col in df.columns:
        # Lọc bỏ giá trị NaN
        col_data = df[col].dropna()
        # Đếm số lần xuất hiện của từng giá trị
        counts = col_data.value_counts()
        # Chỉ lấy các giá trị xuất hiện nhiều hơn 1 lần (trùng lặp)
        duplicates = counts[counts > 1]
        # Gộp tên tiêu đề với dữ liệu và số lượng trùng lặp
        for value, count in duplicates.items():
            try:
                Mon, GV = value.split(" - ")
                merged_data.append([col, Mon, GV, count])
            except Exception as e:
                print(f"Error parsing value '{value}': {e}")
    return merged_data

In [3]:
file_path = '/home/lamkien/PyProjects/TSTKB/FileTKB/T2_HK1.xls'
duplicates_result = ReadSheet(file_path)
print(duplicates_result)

[['10A1', 'TD/GDTC', 'Quang', 6], ['10A1', 'TrNg', 'Xuân', 6], ['10A1', 'TOAN', 'Hướng', 5], ['10A1', 'VAN', 'Lan', 5], ['10A1', 'NN', 'Nhàn', 4], ['10A1', 'LY', 'Ấn', 3], ['10A1', 'QPAN', 'Châm', 2], ['10A1', 'SINH', 'N.Hồng', 2], ['10A1', 'HOA', 'L.Hiền', 2], ['10A1', 'DIA', 'Khoa', 2], ['10A1', 'SU', 'Mến', 2], ['10A2', 'TD/GDTC', 'Tr.Thắng', 6], ['10A2', 'TrNg', 'Giang', 6], ['10A2', 'VAN', 'Trang', 5], ['10A2', 'TOAN', 'Tr.Phương', 5], ['10A2', 'NN', 'Mai', 4], ['10A2', 'SINH', 'N.Hồng', 3], ['10A2', 'TIN', 'Kiên', 2], ['10A2', 'SU', 'Mến', 2], ['10A2', 'HOA', 'L.Hiền', 2], ['10A2', 'DIA', 'Tuyết', 2], ['10A2', 'QPAN', 'Châm', 2], ['10C5', 'TD/GDTC', 'Thiềm', 6], ['10C5', 'TrNg', 'Ng.Phương', 6], ['10C5', 'VAN', 'Lan', 5], ['10C5', 'TOAN', 'Tr.Phương', 5], ['10C5', 'NN', 'H.Hồng', 4], ['10C5', 'SU', 'Mến', 3], ['10C5', 'LY', 'Chúc', 3], ['10C5', 'QPAN', 'Châm', 2], ['10C5', 'CN', 'DuyênB', 2], ['10C5', 'TIN', 'Huyền', 2], ['10C1', 'TD/GDTC', 'Tr.Thắng', 6], ['10C1', 'TrNg', 'Hoa',

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
import math
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Dữ liệu duplicates_result
duplicates_result = [['10A1', 'TD/GDTC', 'Quang', 6], ['10A1', 'TrNg', 'Xuân', 5], ['10A1', 'TOAN', 'Hướng', 6], ['10A1', 'VAN', 'Lan', 6], ['10A1', 'NN', 'Nhàn', 4], ['10A1', 'LY', 'Ấn', 3], ['10A1', 'QPAN', 'Châm', 2], ['10A1', 'SINH', 'N.Hồng', 2], ['10A1', 'HOA', 'L.Hiền', 2], ['10A1', 'DIA', 'Khoa', 2], ['10A1', 'SU', 'Mến', 2], ['10A2', 'TD/GDTC', 'Tr.Thắng', 6], ['10A2', 'TrNg', 'Giang', 6], ['10A2', 'VAN', 'Trang', 5], ['10A2', 'TOAN', 'Tr.Phương', 5], ['10A2', 'NN', 'Mai', 4], ['10A2', 'SINH', 'N.Hồng', 3], ['10A2', 'TIN', 'Kiên', 2], ['10A2', 'SU', 'Mến', 2], ['10A2', 'HOA', 'L.Hiền', 2], ['10A2', 'DIA', 'Tuyết', 2], ['10A2', 'QPAN', 'Châm', 2], ['10C5', 'TD/GDTC', 'Thiềm', 6], ['10C5', 'TrNg', 'Ng.Phương', 6], ['10C5', 'VAN', 'Lan', 5], ['10C5', 'TOAN', 'Tr.Phương', 5], ['10C5', 'NN', 'H.Hồng', 4], ['10C5', 'SU', 'Mến', 3], ['10C5', 'LY', 'Chúc', 3], ['10C5', 'QPAN', 'Châm', 2], ['10C5', 'CN', 'DuyênB', 2], ['10C5', 'TIN', 'Huyền', 2], ['10C1', 'TD/GDTC', 'Tr.Thắng', 6], ['10C1', 'TrNg', 'Hoa', 6], ['10C1', 'VAN', 'Hằng', 5], ['10C1', 'TOAN', 'Giang', 5], ['10C1', 'NN', 'H.Hồng', 4], ['10C1', 'SU', 'L.Phương', 3], ['10C1', 'DIA', 'Nga', 3], ['10C1', 'SINH', 'M.Phương', 2], ['10C1', 'QPAN', 'Châm', 2], ['10C1', 'CN', 'DuyênB', 2], ['10C2', 'TD/GDTC', 'Tr.Thắng', 6], ['10C2', 'TrNg', 'Khánh', 6], ['10C2', 'VAN', 'Hằng', 5], ['10C2', 'TOAN', 'Hướng', 5], ['10C2', 'NN', 'T.Trung', 4], ['10C2', 'SU', 'L.Phương', 3], ['10C2', 'DIA', 'Khoa', 3], ['10C2', 'CN', 'Nhung', 2], ['10C2', 'SINH', 'N.Hồng', 2], ['10C2', 'QPAN', 'Châm', 2], ['10C3', 'TD/GDTC', 'Tr.Thắng', 6], ['10C3', 'TrNg', 'Chúc', 6], ['10C3', 'VAN', 'Lan', 5], ['10C3', 'TOAN', 'Hướng', 5], ['10C3', 'NN', 'Mai', 4], ['10C3', 'SU', 'Mến', 3], ['10C3', 'DIA', 'Tuyết', 3], ['10C3', 'CN', 'Nhung', 2], ['10C3', 'SINH', 'N.Hồng', 2], ['10C3', 'QPAN', 'Châm', 2], ['10C4', 'TD/GDTC', 'Thiềm', 6], ['10C4', 'TrNg', 'M.Phương', 6], ['10C4', 'VAN', 'Trang', 5], ['10C4', 'TOAN', 'Giang', 5], ['10C4', 'NN', 'Nhàn', 4], ['10C4', 'SU', 'L.Phương', 3], ['10C4', 'DIA', 'Nga', 3], ['10C4', 'CN', 'DuyênB', 2], ['10C4', 'QPAN', 'Châm', 2], ['10C4', 'SINH', 'M.Phương', 2], ['11A1', 'TrNg', 'Ái', 6], ['11A1', 'VAN', 'B.Hiền', 5], ['11A1', 'TOAN', 'DuyênA', 5], ['11A1', 'LY', 'Ng.Phương', 4], ['11A1', 'NN', 'H.Hồng', 4], ['11A1', 'HOA', 'L.Hiền', 3], ['11A1', 'TD/GDTC', 'Tr.Thắng', 3], ['11A1', 'SU', 'Mến', 2], ['11A1', 'SINH', 'Hiển', 2], ['11A1', 'QPAN', 'Quang', 2], ['11A1', 'TIN', 'Kiên', 2], ['11A1', 'GDĐP', 'M.Phương', 2], ['11A2', 'TrNg', 'Chúc', 6], ['11A2', 'VAN', 'Lan', 5], ['11A2', 'TOAN', 'Tr.Phương', 5], ['11A2', 'TIN', 'Huyền', 4], ['11A2', 'CN', 'DuyênB', 4], ['11A2', 'NN', 'Đức', 4], ['11A2', 'TD/GDTC', 'Thiềm', 3], ['11A2', 'GDĐP', 'M.Phương', 2], ['11A2', 'DIA', 'Khoa', 2], ['11A2', 'SU', 'L.Phương', 2], ['11A2', 'QPAN', 'Châm', 2], ['11C1', 'TrNg', 'Ấn', 6], ['11C1', 'VAN', 'Trang', 6], ['11C1', 'NN', 'Đức', 4], ['11C1', 'SU', 'L.Phương', 4], ['11C1', 'TOAN', 'DuyênA', 4], ['11C1', 'DIA', 'Khoa', 4], ['11C1', 'TD/GDTC', 'Thiềm', 3], ['11C1', 'CN', 'Ái', 2], ['11C1', 'GDĐP', 'Khánh', 2], ['11C1', 'QPAN', 'Châm', 2], ['11C1', 'SINH', 'Hiển', 2], ['11C2', 'TrNg', 'Đức', 6], ['11C2', 'VAN', 'B.Hiền', 6], ['11C2', 'TOAN', 'DuyênA', 4], ['11C2', 'DIA', 'Nga', 4], ['11C2', 'NN', 'Đức', 4], ['11C2', 'SU', 'L.Phương', 4], ['11C2', 'TD/GDTC', 'Tr.Thắng', 3], ['11C2', 'GDĐP', 'M.Phương', 2], ['11C2', 'SINH', 'Hiển', 2], ['11C2', 'QPAN', 'Châm', 2], ['11C2', 'CN', 'Nhung', 2], ['11C3', 'VAN', 'B.Hiền', 6], ['11C3', 'TrNg', 'Huyền', 6], ['11C3', 'SU', 'Mến', 4], ['11C3', 'NN', 'Đức', 4], ['11C3', 'DIA', 'Tuyết', 4], ['11C3', 'TOAN', 'Khánh', 4], ['11C3', 'TD/GDTC', 'L.Thắng', 3], ['11C3', 'GDĐP', 'Khánh', 2], ['11C3', 'CN', 'Ái', 2], ['11C3', 'QPAN', 'Châm', 2], ['11C3', 'SINH', 'Hiển', 2], ['11C4', 'VAN', 'B.Hiền', 7], ['11C4', 'TrNg', 'Chúc', 6], ['11C4', 'TOAN', 'Hướng', 5], ['11C4', 'SU', 'Mến', 4], ['11C4', 'NN', 'H.Hồng', 4], ['11C4', 'TD/GDTC', 'L.Thắng', 3], ['11C4', 'QPAN', 'Châm', 2], ['11C4', 'GDĐP', 'Quang', 2], ['11C4', 'SINH', 'M.Phương', 2], ['11C4', 'CN', 'Ái', 2], ['12A1', 'TOAN', 'Tiến', 6], ['12A1', 'TrNg', 'Thoa', 6], ['12A1', 'LY', 'Ng.Phương', 5], ['12A1', 'HOA', 'Thoa', 5], ['12A1', 'NN', 'Mai', 4], ['12A1', 'VAN', 'Hằng', 4], ['12A1', 'SINH', 'N.Hồng', 3], ['12A1', 'QPAN', 'Châm', 2], ['12A1', 'TD/GDTC', 'Quang', 2], ['12A1', 'TIN', 'Kiên', 2], ['12C1', 'VAN', 'Trang', 6], ['12C1', 'TrNg', 'L.Thắng', 6], ['12C1', 'DIA', 'Khoa', 5], ['12C1', 'TOAN', 'Giang', 4], ['12C1', 'NN', 'Nhàn', 4], ['12C1', 'SU', 'L.Phương', 3], ['12C1', 'GDKTPL', 'Hiếu', 3], ['12C1', 'SINH', 'Hiển', 3], ['12C1', 'TD/GDTC', 'L.Thắng', 2], ['12C1', 'QPAN', 'Châm', 2], ['12C1', 'TIN', 'Kiên', 2], ['12C2', 'VAN', 'Hằng', 6], ['12C2', 'TrNg', 'Hoa', 6], ['12C2', 'DIA', 'Nga', 5], ['12C2', 'TOAN', 'Giang', 4], ['12C2', 'NN', 'H.Hồng', 4], ['12C2', 'SU', 'Mến', 3], ['12C2', 'SINH', 'N.Hồng', 3], ['12C2', 'GDKTPL', 'Hiếu', 3], ['12C2', 'TD/GDTC', 'Thiềm', 2], ['12C2', 'TIN', 'Huyền', 2], ['12C2', 'QPAN', 'Châm', 2], ['12C3', 'TOAN', 'Khánh', 6], ['12C3', 'TrNg', 'Nhung', 6], ['12C3', 'CN', 'Ái', 5], ['12C3', 'TIN', 'Kiên', 4], ['12C3', 'VAN', 'B.Hiền', 4], ['12C3', 'NN', 'Nhàn', 4], ['12C3', 'DIA', 'Khoa', 3], ['12C3', 'GDKTPL', 'Khuyên', 3], ['12C3', 'QPAN', 'Quang', 2], ['12C3', 'TD/GDTC', 'Quang', 2], ['12C4', 'TOAN', 'Khánh', 6], ['12C4', 'TrNg', 'Thoa', 6], ['12C4', 'CN', 'Nhung', 5], ['12C4', 'VAN', 'Trang', 4], ['12C4', 'NN', 'Mai', 4], ['12C4', 'TIN', 'Huyền', 4], ['12C4', 'DIA', 'Tuyết', 3], ['12C4', 'GDKTPL', 'Hiếu', 3], ['12C4', 'TD/GDTC', 'Thiềm', 2], ['12C4', 'QPAN', 'Quang', 2], ['12A2', 'TrNg', 'Hoa', 6], ['12A2', 'TOAN', 'DuyênA', 6], ['12A2', 'HOA', 'Thoa', 5], ['12A2', 'LY', 'Ng.Phương', 5], ['12A2', 'VAN', 'Lan', 4], ['12A2', 'NN', 'Đức', 4], ['12A2', 'SINH', 'Hiển', 3], ['12A2', 'QPAN', 'Châm', 2], ['12A2', 'TIN', 'Kiên', 2], ['12A2', 'TD/GDTC', 'L.Thắng', 2]]

# Danh sách các môn ưu tiên
priority_subjects = ['TD/GDTC', 'TrNg', 'QPAN']

# Chuẩn bị dữ liệu
classes = list(set([row[0] for row in duplicates_result]))
teachers = list(set([row[2] for row in duplicates_result]))
subjects = list(set([row[1] for row in duplicates_result]))

class_idx = {c: i for i, c in enumerate(classes)}
teacher_idx = {t: i for i, t in enumerate(teachers)}
subject_idx = {s: i for i, s in enumerate(subjects)}

num_rows = len(duplicates_result)
num_classes = len(classes)
num_teachers = len(teachers)
num_subjects = len(subjects)

# Chuẩn hóa dữ liệu đặc trưng thủ công
features = []
for row in duplicates_result:
    class_id = class_idx[row[0]]
    subject_id = subject_idx[row[1]]
    teacher_id = teacher_idx[row[2]]
    periods = int(row[3])
    features.append([class_id, subject_id, teacher_id, periods])
features_tensor = torch.tensor(features, dtype=torch.float32)
mean = features_tensor.mean(dim=0, keepdim=True)
std = features_tensor.std(dim=0, keepdim=True) + 1e-6
features_tensor = (features_tensor - mean) / std

# Tạo tensor số tiết
periods_tensor = torch.tensor([int(row[3]) for row in duplicates_result], dtype=torch.float32)  # [seq_len]

# Tạo ma trận ánh xạ dòng -> lớp, dòng -> giáo viên
row_class = torch.tensor([class_idx[row[0]] for row in duplicates_result])
row_teacher = torch.tensor([teacher_idx[row[2]] for row in duplicates_result])

# Xác định các môn ưu tiên
is_priority = torch.tensor([1 if row[1] in priority_subjects else 0 for row in duplicates_result], dtype=torch.float32)  # [seq_len]

# Tính tổng số tiết của mỗi giáo viên
teacher_periods = defaultdict(int)
for row in duplicates_result:
    teacher_periods[row[2]] += int(row[3])

# Xác định giáo viên ít tiết (dưới 10 tiết)
low_period_teachers = {t for t, p in teacher_periods.items() if p < 10}
low_period_teacher_mask = torch.tensor([1 if row[2] in low_period_teachers else 0 for row in duplicates_result], dtype=torch.float32)

# Số tiết tối đa
max_morning = 20
max_afternoon = 15
max_class_morning = 30

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# Mô hình Transformer
class TransformerScheduler(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, dropout=0.1):
        super().__init__()
        self.input_linear = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_model * 4, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.output_linear = nn.Linear(d_model, 1)  # Dự đoán số tiết sáng

    def forward(self, x, periods_tensor, is_priority):
        x = self.input_linear(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        morning = self.output_linear(x).squeeze(-1)  # [batch_size, seq_len]
        morning = torch.clamp(morning, min=0)
        morning = torch.minimum(morning, periods_tensor)
        morning = torch.round(morning)
        afternoon = periods_tensor - morning

        # Ưu tiên môn TD/GDTC, TrNg, QPAN
        for i in range(morning.size(1)):
            if is_priority[i].item() == 1:
                total_periods = periods_tensor[i].item()
                if total_periods == 6:
                    dist_to_0 = torch.abs(morning[:, i])
                    dist_to_3 = torch.abs(morning[:, i] - 3)
                    dist_to_6 = torch.abs(morning[:, i] - 6)
                    min_dist, idx = torch.min(torch.stack([dist_to_0, dist_to_3, dist_to_6]), dim=0)
                    morning[:, i] = torch.where(idx == 0, 0, torch.where(idx == 1, 3, 6))
                    afternoon[:, i] = total_periods - morning[:, i]
                elif total_periods in [2, 3]:
                    dist_to_0 = torch.abs(morning[:, i])
                    dist_to_max = torch.abs(morning[:, i] - total_periods)
                    morning[:, i] = torch.where(dist_to_0 < dist_to_max, 0, total_periods)
                    afternoon[:, i] = total_periods - morning[:, i]

        return morning, afternoon

# Khởi tạo mô hình
d_model = 128
nhead = 8
num_layers = 4
input_dim = 4
model = TransformerScheduler(input_dim, d_model, nhead, num_layers, dropout=0.2)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=100)  # Bỏ verbose

# Hàm mất mát
def compute_loss(morning, afternoon, periods_tensor, is_priority, row_teacher, row_class):
    loss = 0.0
    loss += ((morning + afternoon - periods_tensor) ** 2).sum() * 500
    for t in range(num_teachers):
        mask = (row_teacher == t)
        loss += torch.relu(morning[mask].sum() - max_morning) * 1000
        loss += torch.relu(afternoon[mask].sum() - max_afternoon) * 1000
    for c in range(num_classes):
        mask = (row_class == c)
        loss += torch.relu(morning[mask].sum() - max_class_morning) * 1000
    for t in low_period_teachers:
        t_idx = teacher_idx[t]
        mask = (row_teacher == t_idx)
        morning_sum = morning[mask].sum()
        afternoon_sum = afternoon[mask].sum()
        loss += torch.abs(morning_sum * afternoon_sum) * 500
    for i in range(num_rows):
        if is_priority[i].item() == 1:
            total_periods = periods_tensor[i].item()
            if total_periods == 6:
                dist_to_0 = torch.abs(morning[i])
                dist_to_3 = torch.abs(morning[i] - 3)
                dist_to_6 = torch.abs(morning[i] - 6)
                min_dist = torch.min(torch.stack([dist_to_0, dist_to_3, dist_to_6]))
                loss += min_dist * 200
            elif total_periods in [2, 3]:
                dist_to_0 = torch.abs(morning[i])
                dist_to_max = torch.abs(morning[i] - total_periods)
                min_dist = torch.min(torch.stack([dist_to_0, dist_to_max]))
                loss += min_dist * 200
    return loss

# Huấn luyện mô hình
features_tensor = features_tensor.unsqueeze(0)  # [1, seq_len, input_dim]
num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    morning, afternoon = model(features_tensor, periods_tensor, is_priority)
    loss = compute_loss(morning.squeeze(0), afternoon.squeeze(0), periods_tensor, is_priority, row_teacher, row_class)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step(loss)
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}, LR: {optimizer.param_groups[0]["lr"]}')

# Kết quả cuối cùng
morning, afternoon = model(features_tensor, periods_tensor, is_priority)
morning = morning.squeeze(0)  # [seq_len]
afternoon = afternoon.squeeze(0)  # [seq_len]
result = []
for i, row in enumerate(duplicates_result):
    result.append([row[0], row[1], row[2], int(morning[i].item()), int(afternoon[i].item())])

# Nhóm kết quả theo lớp
grouped_result = defaultdict(list)
for r in result:
    class_name = r[0]
    grouped_result[class_name].append(r)

# In kết quả nhóm theo lớp
print("\nKết quả phân bổ tiết học (nhóm theo lớp):")
for class_name in sorted(grouped_result.keys()):
    print(f"\nLớp: {class_name}")
    print("Môn\t\tGiáo viên\tSáng\tChiều")
    print("-" * 40)
    for r in grouped_result[class_name]:
        subject = r[1].ljust(8)  # Căn lề môn học
        teacher = r[2].ljust(10)  # Căn lề giáo viên
        morning = r[3]
        afternoon = r[4]
        print(f"{subject}\t{teacher}\t{morning}\t{afternoon}")

# Kiểm tra giáo viên ít tiết
print("\nGiáo viên ít tiết và phân bố tiết:")
for t in sorted(low_period_teachers):
    t_idx = teacher_idx[t]
    mask = (row_teacher == t_idx)
    morning_sum = morning[mask].sum().item()
    afternoon_sum = afternoon[mask].sum().item()
    print(f"{t}: Sáng {morning_sum}, Chiều {afternoon_sum}")

Epoch 0, Loss: 180000.0, LR: 0.0005
Epoch 100, Loss: 180000.0, LR: 0.0005
Epoch 200, Loss: 180000.0, LR: 0.00025
Epoch 300, Loss: 180000.0, LR: 0.000125
Epoch 400, Loss: 180000.0, LR: 6.25e-05
Epoch 500, Loss: 180000.0, LR: 3.125e-05
Epoch 600, Loss: 180000.0, LR: 1.5625e-05
Epoch 700, Loss: 180000.0, LR: 7.8125e-06
Epoch 800, Loss: 180000.0, LR: 3.90625e-06
Epoch 900, Loss: 180000.0, LR: 1.953125e-06
Epoch 1000, Loss: 180000.0, LR: 9.765625e-07
Epoch 1100, Loss: 180000.0, LR: 4.8828125e-07
Epoch 1200, Loss: 180000.0, LR: 2.44140625e-07
Epoch 1300, Loss: 180000.0, LR: 1.220703125e-07
Epoch 1400, Loss: 180000.0, LR: 6.103515625e-08
Epoch 1500, Loss: 180000.0, LR: 3.0517578125e-08
Epoch 1600, Loss: 180000.0, LR: 1.52587890625e-08
Epoch 1700, Loss: 180000.0, LR: 1.52587890625e-08
Epoch 1800, Loss: 180000.0, LR: 1.52587890625e-08
Epoch 1900, Loss: 180000.0, LR: 1.52587890625e-08
Epoch 2000, Loss: 180000.0, LR: 1.52587890625e-08
Epoch 2100, Loss: 180000.0, LR: 1.52587890625e-08
Epoch 2200, 

TypeError: 'int' object is not subscriptable